# Objective 

In [1]:
import spatialdata as sd
from pathlib import Path
from rna_seg.dataset_zarr.patches import create_patch_rnaseg
from rna_seg.dataset_zarr.RNAsegDataset import RNAsegDataset, custom_collate_fn
from rna_seg.models import RNASeg
import numpy as np
import albumentations as A
import cv2
import torch
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from rna_seg.train import train_one_epoch

/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


No module named 'vmunet'
VMUnet not loaded


### step 1 create training dataset from zarr files
* create crops shape 
* create folder of csv

In [2]:
### load sdata and set path parameters 
merfish_zarr_path = "/media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr"
sdata = sd.read_zarr(merfish_zarr_path)
image_key = "staining_z3"
patch_width = 1200
patch_overlap = 50
points_key = "transcripts"
min_transcripts_per_patch = 0
folder_patch_rna_seg = Path(merfish_zarr_path) / ".rna_seg"

### create patch in the sdata and precompute transcipt.csv for each patch with sopa
create_patch_rnaseg(sdata,
                    image_key=image_key,
                    points_key=points_key,
                    patch_width=patch_width,
                    patch_overlap=patch_overlap,
                    min_transcripts_per_patch=min_transcripts_per_patch,
                    overwrite = True)
print(sdata)

/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/spatialdata/_core/_elements.py:96: UserWarning: Key `sopa_patches_rna_seg_1200_50` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)
[INFO] (sopa.patches.patches) 64 patches were saved in sdata['sopa_patches_rna_seg_1200_50']


[########################################] | 100% Completed | 19.08 ss
SpatialData object, with associated Zarr store: /media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr
├── Images
│     └── 'staining_z3': DataTree[cyx] (5, 9000, 9000), (5, 4500, 4500), (5, 2250, 2250), (5, 1125, 1125), (5, 562, 562)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
└── Shapes
      ├── 'Cellbound1': GeoDataFrame shape: (3258, 1) (2D shapes)
      ├── 'Cellbound1_consistent_with_nuclei': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'Cellbound1_consistent_without_nuclei': GeoDataFrame shape: (2239, 1) (2D shapes)
      ├── 'DAPI': GeoDataFrame shape: (2377, 1) (2D shapes)
      ├── 'DAPI_consistent_in_cell': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'DAPI_consistent_not_in_cell': GeoDataFrame shape: (1370, 1) (2D shapes)
      ├── 'sopa_patches_rna_seg_1200_50': GeoDataFrame shape: (64, 3) (2D shapes)
      └── 'sopa

In [3]:
sdata

SpatialData object, with associated Zarr store: /media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr
├── Images
│     └── 'staining_z3': DataTree[cyx] (5, 9000, 9000), (5, 4500, 4500), (5, 2250, 2250), (5, 1125, 1125), (5, 562, 562)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
└── Shapes
      ├── 'Cellbound1': GeoDataFrame shape: (3258, 1) (2D shapes)
      ├── 'Cellbound1_consistent_with_nuclei': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'Cellbound1_consistent_without_nuclei': GeoDataFrame shape: (2239, 1) (2D shapes)
      ├── 'DAPI': GeoDataFrame shape: (2377, 1) (2D shapes)
      ├── 'DAPI_consistent_in_cell': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'DAPI_consistent_not_in_cell': GeoDataFrame shape: (1370, 1) (2D shapes)
      ├── 'sopa_patches_rna_seg_1200_50': GeoDataFrame shape: (64, 3) (2D shapes)
      └── 'sopa_patches_rna_seg_1200_150': GeoDataFrame shape: (81, 3) (2D shapes)
wit

### Step 2 create training dataset from sdata

In [6]:

key_shape_cell_seg = "Cellbound1"
key_nuclei_segmentation="DAPI"
# to name for future shape that will be created in the sdata
key_cell_consistent = "Cellbound1_consistent"
key_nucleus_consistent = "DAPI_consistent"

transform_resize  = A.Compose([
 A.Resize(width=512, height=512, interpolation=cv2.INTER_NEAREST),
])


dataset = RNAsegDataset(
 sdata=sdata,
 channels_dapi= ["DAPI"],
 channels_cellbound=["Cellbound1"],

 shape_patch_key="sopa_patches_rna_seg_1200_50",

 key_cell_consistent=key_cell_consistent,
    key_nucleus_consistent=key_nucleus_consistent,
 key_nuclei_segmentation=key_nuclei_segmentation,
 gene_column="gene",
 density_threshold = None,
 kernel_size_background_density = 10 ,
 kernel_size_rna2img = 0.5,
 max_filter_size_rna2img = 2,
 transform_resize = transform_resize,
    training_mode = True,
    path_cache = folder_patch_rna_seg,
     patch_width=1200,
     patch_overlap=50,
 )

100%|██████████| 64/64 [00:03<00:00, 18.66it/s]
Number of valid patches: 48


### set threshold on density

In [7]:
dataset.set_threshold( max_nb_crops=500,
              kernel_size = 9,
              percentile_threshold = 5,
              shape=(1200, 1200),)
dataset[0].keys()

100%|██████████| 64/64 [00:02<00:00, 23.85it/s]
compute density threshold


100%|██████████| 48/48 [00:08<00:00,  5.56it/s]


Time to compute density threshold: 8.630728s
Error while loading cache: [Errno 2] No such file or directory: '/media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr/.rna_seg/0/img_cellbound.tif'
Recomputing the patch 0


UnboundLocalError: local variable 'remove_cell_in_background' referenced before assignment

# Step 3: Initilize and RNAseg Model, optinally load pretrained weights

In [ ]:
device="cpu"

rnaseg = RNASeg(device,
                net='unet',
                flow_threshold = 0.9,
                cellbound_flow_threshold = 0.4,
                pretrained_model = None,
                )
rnaseg.to(device)
print('ok')

In [ ]:
validation_split = 0.1
indices = list(range(len(dataset)))
split = int(np.floor(validation_split * len(dataset)))

np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_indices) # does it change the order at each iteration?
valid_sampler = SubsetRandomSampler(val_indices) # does it change the order at each iteration?


#dataset.reloaded = False
training_loader = torch.utils.data.DataLoader(dataset,
                                              batch_size=2,
                                              shuffle=False,
                                              num_workers = 5,
                                              sampler=train_sampler,
                                              collate_fn = custom_collate_fn,
                                              )

print( f"len(training_loader) {len(training_loader)}")

validation_loader = torch.utils.data.DataLoader(dataset,
                                                batch_size=2,
                                                shuffle=False,
                                                num_workers =  3,
                                                sampler=valid_sampler,
                                                collate_fn = custom_collate_fn,
                                                )

optimizer = torch.optim.AdamW(rnaseg.parameters(),
                              lr=0.001,
                              weight_decay=0.01)

                                

In [13]:
best_val_loss = np.inf
path_save_model = "/home/tom/toremove"
for epoch_index in tqdm(range(3)):

    train_one_epoch(
        device=device,
    epoch_index=epoch_index,
    rnaseg=rnaseg,
    training_loader=training_loader,
    optimizer=optimizer,
    print_loss_every = int(len(training_loader) /3),
    tb_writer= None,
    validation_loader=validation_loader,
    path_save_model=path_save_model,
    cellbound_prob=  0.8,
    best_val_loss=best_val_loss
    )

  0%|          | 0/3 [00:00<?, ?it/s]


batch 0  0%|          | 0/22 [00:00<?, ?it/s]
  batch nb 1 loss: 1.2506969996861048 
batch 0
batch 1
  validation loss: 6.478799104690552
best_val_loss: 6.478799104690552

batch 1  5%|▍         | 1/22 [00:11<04:03, 11.61s/it]

batch 2  9%|▉         | 2/22 [00:20<03:22, 10.11s/it]

batch 3 14%|█▎        | 3/22 [00:26<02:36,  8.25s/it]

batch 4 18%|█▊        | 4/22 [00:32<02:10,  7.25s/it]

batch 5 23%|██▎       | 5/22 [00:38<01:57,  6.90s/it]

batch 6 27%|██▋       | 6/22 [00:44<01:44,  6.52s/it]

batch 7 32%|███▏      | 7/22 [00:49<01:31,  6.07s/it]
  batch nb 8 loss: 8.296125854764666 
batch 0
batch 1
  validation loss: 5.849609136581421
best_val_loss: 5.849609136581421

batch 8 36%|███▋      | 8/22 [00:58<01:39,  7.11s/it]

batch 9 41%|████      | 9/22 [01:04<01:27,  6.69s/it]

batch 1045%|████▌     | 10/22 [01:10<01:15,  6.30s/it]

batch 1150%|█████     | 11/22 [01:15<01:05,  5.94s/it]

batch 1255%|█████▍    | 12/22 [01:20<00:56,  5.65s/it]

batch 1359%|█████▉    | 13/22 [01:25<00:

 33%|███▎      | 1/3 [02:21<04:43, 141.69s/it]



batch 0  0%|          | 0/22 [00:00<?, ?it/s]
  batch nb 1 loss: 0.644050053187779 
batch 0
batch 1
  validation loss: 4.956424713134766
best_val_loss: 4.956424713134766

batch 1  5%|▍         | 1/22 [00:09<03:21,  9.60s/it]

batch 2  9%|▉         | 2/22 [00:14<02:19,  7.00s/it]

batch 3 14%|█▎        | 3/22 [00:20<02:00,  6.35s/it]

batch 4 18%|█▊        | 4/22 [00:25<01:44,  5.83s/it]

batch 5 23%|██▎       | 5/22 [00:30<01:35,  5.63s/it]

batch 6 27%|██▋       | 6/22 [00:36<01:29,  5.59s/it]

batch 7 32%|███▏      | 7/22 [00:41<01:21,  5.46s/it]
  batch nb 8 loss: 6.86816109929766 
batch 0
batch 1
  validation loss: 4.480439066886902
best_val_loss: 4.480439066886902

batch 8 36%|███▋      | 8/22 [00:51<01:35,  6.85s/it]

batch 9 41%|████      | 9/22 [00:56<01:23,  6.45s/it]

batch 1045%|████▌     | 10/22 [01:01<01:12,  6.00s/it]

batch 1150%|█████     | 11/22 [01:06<01:03,  5.75s/it]

batch 1255%|█████▍    | 12/22 [01:12<00:55,  5.55s/it]

batch 1359%|█████▉    | 13/22 [01:17<00:4

 67%|██████▋   | 2/3 [04:40<02:19, 139.83s/it]



batch 0  0%|          | 0/22 [00:00<?, ?it/s]
  batch nb 1 loss: 0.6761629240853446 
batch 0
batch 1
  validation loss: 3.8062593936920166
best_val_loss: 3.8062593936920166

batch 1  5%|▍         | 1/22 [00:10<03:34, 10.20s/it]

batch 2  9%|▉         | 2/22 [00:15<02:31,  7.57s/it]

batch 3 14%|█▎        | 3/22 [00:21<02:03,  6.48s/it]

batch 4 18%|█▊        | 4/22 [00:26<01:48,  6.05s/it]

batch 5 23%|██▎       | 5/22 [00:31<01:36,  5.70s/it]

batch 6 27%|██▋       | 6/22 [00:36<01:27,  5.48s/it]

batch 7 32%|███▏      | 7/22 [00:41<01:21,  5.43s/it]
  batch nb 8 loss: 4.402904902185712 
batch 0
batch 1
  validation loss: 3.7692790031433105
best_val_loss: 3.7692790031433105

batch 8 36%|███▋      | 8/22 [00:51<01:34,  6.76s/it]

batch 9 41%|████      | 9/22 [00:57<01:22,  6.35s/it]

batch 1045%|████▌     | 10/22 [01:02<01:11,  5.95s/it]

batch 1150%|█████     | 11/22 [01:07<01:02,  5.66s/it]

batch 1255%|█████▍    | 12/22 [01:12<00:54,  5.44s/it]

batch 1359%|█████▉    | 13/22 [01:1

100%|██████████| 3/3 [06:54<00:00, 138.02s/it]